## Preprocess data to fit model format

In [1]:
import pandas as pd 
import numpy as np
print(pd.__version__)

1.1.3


In [2]:
def get_script_from_id(id):
    script = open('../data/script/' + id + '.script', 'r').read()
    # print(script)
    script = script.replace("'", " ").replace('"', ' ').replace('\n', ' ').replace('\r', ' ').replace('\t', ' ').replace('\b', ' ').replace('\\', ' ')
    return script

In [3]:
inputFile = open('../data_gathering/baseline/output/imdb_id_with_age_rating_and_labels.txt')
df_data = []
for line in inputFile:
    line_data = line.strip().split(',')
    # print(line_data)
    line_data.append(int(line_data[3]) + int(line_data[4]) + int(line_data[5]) + int(line_data[6]))
    
    max_index = 0
    max_value = 0
    for i in range(3,7):
        vote_count = int(line_data[i])
        if(vote_count >= max_value):
            max_index = i - 3
            max_value = vote_count
    line_data.append(max_index)
    try:
        script = get_script_from_id(line_data[0])
    except:
        #print('Error on loading script for id: ' + line_data[0])
        continue
    line_data.append(script)
    df_data.append(line_data)

# id | Aspect | None | Mild | Moderate | Severe | Total_votes | Aspect_rating | text
df = pd.DataFrame(df_data, columns=['id', 'age_rating', 'Aspect', 'None', 'Mild', 'Moderate', 'Severe', 'Total_votes', 'Aspect_rating', 'text'])
df.drop(columns=["age_rating"], inplace=True)
df.reset_index(drop=True, inplace=True)
df = df.astype({'Mild':'int', 'Moderate':'int', 'Severe':'int', 'None':'int', 'Total_votes':'int', 'Aspect_rating':'int'})
df.head()


,id,Aspect,None,Mild,Moderate,Severe,Total_votes,Aspect_rating,text
0,tt0032138,nudity,93,3,0,8,104,0,FADE IN -- Title: For nearly forty years this...
1,tt0032138,violence,23,61,8,8,100,1,FADE IN -- Title: For nearly forty years this...
2,tt0032138,profanity,86,4,0,5,95,0,FADE IN -- Title: For nearly forty years this...
3,tt0032138,alcohol,76,10,1,6,93,0,FADE IN -- Title: For nearly forty years this...
4,tt0032138,frightening,10,63,17,10,100,1,FADE IN -- Title: For nearly forty years this...


In [4]:
import pandas as pd
import pickle

def map_data_frame_to_pickle(data_frame, file_name, path='./'):
    file_name = f'{file_name}.pkl'
    with open(path + file_name, 'wb') as f:
        data_frame.to_pickle(f, protocol=4)

def split_df_into_test_and_train(aspect_name, df, path='./'):
    df_train = df.sample(frac=0.85, random_state=0)
    df_test = df.drop(df_train.index)
    map_data_frame_to_pickle(df_train, f'{aspect_name}_train', '../data/pickle/')
    map_data_frame_to_pickle(df_test, f'{aspect_name}_test', '../data/pickle/')
    map_data_frame_to_pickle(df_test, f'{aspect_name}_dev', '../data/pickle/')




In [5]:
# group the dataframe by 'aspect' and create a dictionary of dataframes
df_dict = {aspect: aspect_df.drop('Aspect', axis=1) for aspect, aspect_df in df.groupby('Aspect')}

# print the dictionary of dataframes
for aspect, aspect_df in df_dict.items():
    print(aspect, len(aspect_df))
    split_df_into_test_and_train(aspect, aspect_df, '../data/pickle/')


alcohol 478
frightening 479
nudity 479
profanity 478
violence 478


In [8]:
 # Test the pickle generation

import pickle as pkl
import pandas as pd

 # map_data_frame_to_pickle(df, 'train_emb')

with open("../data_gathering/baseline/output/imdb_id_with_embSentencesList.pkl", "rb") as f:
    object = pkl.load(f)
    
df = pd.DataFrame(object)
print(df)
#print(df['text_emb'][78].shape)
#df.to_csv(r'train_emb.csv')

     imdb_id                                      sentences_emb
0  tt0032138  [[tensor(0.1276), tensor(-0.1368), tensor(0.79...
1  tt0035423  [[tensor(-0.6777), tensor(0.5362), tensor(0.49...
2  tt0038650  [[tensor(0.1077), tensor(-0.2843), tensor(0.59...
3  tt0047396  [[tensor(-0.9443), tensor(-0.1430), tensor(0.1...
4  tt0048545  [[tensor(0.3541), tensor(0.8298), tensor(0.594...
5  tt0049470  [[tensor(-0.6712), tensor(0.4965), tensor(0.08...
